# FUNKTION RESULTATWANDLER

Diese Funktion nimmt Tennis-Resultate von der ATP Homepage entgegen und wandelt sie in ein maschinen-lesbares Format um. So können wir später Berechnungen anstellen.

winlose = ""
...
...
resultat = spiele.find_all('td')[4].sub.replace_with('')
resultat = resultat.text.strip()
score = resultatwandler(resultat)

In [39]:
def resultatwandler(original): # bitte im Format '64 63' oder '(W/O)' an Funktion übergeben
    output = ""
    if original == "": # Bye
        output = "BYE"
    elif original == "(W/O)": # Walkover
        output = "WO"
    elif "(RET)" in original: # Retired
        output = "RET"
    else: # Resultate wurden gespielt
        saetze = original.split(" ") # Teile das Spiel in einzelne Sätze auf
        output = []
        for satz in saetze:
            if len(satz) == 2: # Wenn es ein normales Satzergebnis ist
                output.append([int(satz[0]), int(satz[1])]) # Achtung: Noch in eine Zahl verwandeln mit int()
            elif "-" in satz: # Wenn es ein Satz ohne Tiebreak war...
                games = satz.split("-")
                output.append([int(games[0]), int(games[1])])
                
                
    # Und gib das nun zurück
    return output

## Testen wir das mit ein paar möglichen Resultaten 


In [45]:
resultatwandler("64 63")

[[6, 4], [6, 3]]

In [46]:
resultate = []
resultate.append(resultatwandler("64 63"))
resultate.append(resultatwandler("64 64 63"))
resultate.append(resultatwandler("36 64 19-17"))
resultate.append(resultatwandler("62 63 36 75"))
resultate.append(resultatwandler("64 63 26 57 64"))
resultate.append(resultatwandler("64 63 26 57 13-11"))
resultate.append(resultatwandler("61 (RET)"))
resultate.append(resultatwandler("60 33 (RET)"))
resultate.append(resultatwandler("(W/O)"))
resultate.append(resultatwandler(""))


## Werten wir aus...

In [47]:
anz_ret = 0
anz_wo = 0
anz_bye = 0

anz_gewonnene_games = 0
anz_verlorene_games = 0

alle_satzresultate = []

for result in resultate:
    #print(result)
    if result == 'RET':
        anz_ret += 1
    elif result == "WO":
        anz_wo += 1
    elif result == "BYE":
        anz_bye += 1
    else:
        # Hier kannst du die Resultate auswerten:
        for satz in result:
            anz_gewonnene_games += satz[0] # Addiere die erste Zahl zu den gewonnenen...
            anz_verlorene_games += satz[1] # ... und die zweite Zahl zu den verlorenen Games
            alle_satzresultate.append(satz)

# Ausgabe von Auswertung
print("Anzahl retired Games: "+str(anz_ret))
print("Anzahl w/o Games: "+str(anz_wo))
print("Anzahl bye Games: "+str(anz_bye))
print("Anzahl gewonnene Games: "+str(anz_gewonnene_games))
print("Anzahl verlorene Games: "+str(anz_verlorene_games))

Anzahl retired Games: 2
Anzahl w/o Games: 1
Anzahl bye Games: 1
Anzahl gewonnene Games: 137
Anzahl verlorene Games: 116


## Werten wir aus (zum Zweiten), diesmal etwas komplizierter

In [48]:
# Um die häufigsten Satzresultate herauszufinden, verwenden wir besser Pandas
import pandas as pd

df_alle_satzresultate = pd.DataFrame(alle_satzresultate) # Mache aus meiner Liste ein DataFrame
df_alle_satzresultate.rename(columns={0:'federer'}, inplace=True) # Benenne die Spalten clever ;-)
df_alle_satzresultate.rename(columns={1:'gegner'}, inplace=True)

df_alle_satzresultate.head(10) # So sieht das dann etwa aus

,federer,gegner
0,6,4
1,6,3
2,6,4
3,6,4
4,6,3
5,3,6
6,6,4
7,19,17
8,6,2
9,6,3


In [49]:
# Wir gruppieren die beiden Spalten und sortieren nach Anzahl Vorkommnissen (das ist so eine Art value_counts(), 
# aber nicht nur nach einer Spalte, sondern nach der Kombination aus unseren beiden Spalten 'federer' und 'gegner')
df_alle_satzresultate.groupby(['federer', 'gegner']).size().sort_values(ascending=False)

federer  gegner
6        4         7
         3         5
5        7         2
3        6         2
2        6         2
19       17        1
13       11        1
7        5         1
6        2         1
dtype: int64